In [41]:
import spacy
nlp = spacy.load('en_core_web_sm')

def get_passage(location) -> list:
    # reads the file and returns a list of sentences
    with open(location, 'r', encoding='utf8') as f:
        passage = f.read()
        passage = passage.replace('\n', ' ')
        passage = passage.replace('    ', ' ')
        passage = passage.replace('    ', ' ')
        passage = nlp(passage)

    #split the file into a list of sentences using spacy
    sentences = list(passage.sents)

    return sentences

In [42]:
path = "text/"
passages = ["protagonist-first-paragraph.txt", "antagonist.txt", "crimescene.txt", "resolution.txt", "evidence.txt"]

sentences = []

for passage in passages:
    sentences.append(get_passage(path + passage))



In [43]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate(text, src, target):
    tokenizer.src_lang = src
    encoded_hi = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id(target))
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


# print(translate("Hello, my dog is cute", "en", "es"))
translated_sp = []
translated_sw = []
for passage in sentences:
    translated_sp.append([])
    translated_sw.append([])
    for sentence in passage:
        translated_sp[-1].append(translate(translate(sentence.text, "en", "es"), "es", "en")) #translate from english to spanish and then spanish to english
        translated_sw[-1].append(translate(translate(sentence.text, "en", "sw"), "sw", "en")) #translate from english to swahili and then swahili to english


In [44]:
#save both the original and translated sentences to a file in translationout
# with open("translationout/original.txt", "w", encoding="utf8") as f:
#     for i in range(len(translated)):
#         f.write(sentences[i].text + "\n")

# with open("translationout/translated.txt", "w", encoding="utf8") as f:
#     for i in range(len(translated)):
#         f.write(translated[i][0] + "\n")

for i in range(len(translated_sp)): #store original
    with open("translationout/original" + str(i) + ".txt", "w", encoding="utf8") as f:
        for j in range(len(translated_sp[i])):
            f.write(sentences[i][j].text + "\n")

for i in range(len(translated_sp)): #store spanish
    with open("translationout/translated_sp_" + i, "w", encoding="utf8") as f:
        for j in range(len(translated_sp[i])):
            f.write(translated_sp[i][j][0] + "\n")

for i in range(len(translated_sw)): #store swahili
    with open("translationout/translated_sw_" + i, "w", encoding="utf8") as f:
        for j in range(len(translated_sw[i])):
            f.write(translated_sw[i][j][0] + "\n")